# Implementing custom Unet training
![](https://drive.google.com/uc?id=1iU9xphUXLys7Dpi7FwSDy1YLCKutjUTQ)

In [1]:
import os
import time
import torch

from PIL import Image

from torch import nn
from torchvision import transforms

from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, AdamW, SGD

In [2]:
class SegmentationDataset(Dataset):
  def __init__(self, image_dir, mask_dir):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor()])

    valid_extension = {".jpg", ".jpeg", ".png"}
    self.images = [f for f in os.listdir(image_dir) if os.path.splitext(f)[1].lower() in valid_extension]

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = os.path.join(self.image_dir, self.images[idx])
    name, ext = os.path.splitext(self.images[idx])
    mask_path = os.path.join(self.mask_dir, f"{name}.png")

    image = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")

    image = self.transform(image)
    mask = self.transform(mask)

    mask = (mask > 0.5).float()

    return image, mask

In [3]:
def get_dataloader(image_dir, maks_dir, batch_size=2, shuffle=True):
  dataset = SegmentationDataset(image_dir, maks_dir)
  return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# UNET

![](https://drive.google.com/uc?id=1-VqrLBuLuzcIxk3QNTIHP5u1JW93NNy5)

In [4]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.conv_op = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3,padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3,padding=1),
        nn.ReLU(inplace=True)
    )

  def forward(self, x):
    return self.conv_op(x)

In [5]:
class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
      super().__init__()
      self.conv = DoubleConv(in_channels, out_channels)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
      down = self.conv(x)
      p = self.pool(down)

      return down, p

In [6]:
class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
       x1 = self.up(x1)
       x = torch.cat([x1, x2], 1)
       return self.conv(x)

In [7]:
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)

        self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
       down_1, p1 = self.down_convolution_1(x)
       down_2, p2 = self.down_convolution_2(p1)
       down_3, p3 = self.down_convolution_3(p2)
       down_4, p4 = self.down_convolution_4(p3)

       b = self.bottle_neck(p4)

       up_1 = self.up_convolution_1(b, down_4)
       up_2 = self.up_convolution_2(up_1, down_3)
       up_3 = self.up_convolution_3(up_2, down_2)
       up_4 = self.up_convolution_4(up_3, down_1)

       out = self.out(up_4)
       return out

In [8]:
class DiceLoss(nn.Module):
  def __init__(self, smooth=1e-6):
    super(DiceLoss, self).__init__()
    self.smooth = smooth

  def forward(self, inputs, targets):
    inputs = inputs.view(-1)
    targets = targets.view(-1)

    intersection = (inputs * targets).sum()
    dice_score = (2. * intersection + self.smooth) / (inputs.sum() + targets.sum() + self.smooth)

    return 1 - dice_score

In [9]:
class BCEWithDiceLoss(nn.Module):
  def __init__(self, smooth=1e-6):
    super(BCEWithDiceLoss, self).__init__()
    self.bce = nn.BCEWithLogitsLoss()
    self.dice = DiceLoss()

  def forward(self, inputs,targets):
    bce_loss = self.bce(inputs,targets)
    dice_loss = self.dice(inputs,targets)
    return 0.5*bce_loss + dice_loss

In [10]:
# Training loop

def train(model, dataloader, epochs=2, lr=0.001, save_path="unet_model", load_path=None):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  if load_path and os.path.exists(load_path):
    print(f"Loading model weights from {load_path}")
    model.load_state_dict(torch.load(load_path, map_loacation=device))
  else:
    print(f"No checkpoint found, training from scratch.")

  print(device)
  model.to(device)

  criterion = BCEWithDiceLoss()
  # criterion = nn.BCEWithLogitsLoss()

  optimizer = SGD(model.parameters(), lr=lr)

  for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for images, masks in dataloader:
      images, masks = images.to(device), masks.to(device)
      optimizer.zero_grad()

      output = model(images)

      loss = criterion(output, masks)
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss : {avg_loss:.4f}, LR : {lr}")

    if epoch % 10 == 0 and epoch>0:
      torch.save(model.state_dict(), f"{save_path}.pth")

  torch.save(model.state_dict(), f"{save_path}_final.pth")
  print(f"Model Saved to {save_path}")


In [11]:
dataloader = get_dataloader("D:\\Projects\\Segmentation\\007_Implementing_custom_Unet_training\\Human-Segmentation-Dataset\\Training_Images", "D:\\Projects\\Segmentation\\007_Implementing_custom_Unet_training\\Human-Segmentation-Dataset\\Ground_Truth", batch_size=8, shuffle=True)

In [ ]:
model = UNet(in_channels=3, num_classes=1)

: 

In [ ]:
train(model, dataloader, epochs=80, lr=0.001)

No checkpoint found, training from scratch.
cpu
Epoch [1/80], Loss : 1.2724, LR : 0.001


## Inference on trained model

In [ ]:
import numpy as np

# Load model and predict with stats
def predict(model_path, input_image_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load model
    model = UNet(in_channels=3, num_classes=1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Track start time
    total_start_time = time.time()

    # Image preprocessing
    preprocess_start_time = time.time()
    image = Image.open(input_image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])
    image_tensor = transform(image).unsqueeze(0).to(device)
    preprocess_end_time = time.time()

    # Model inference
    inference_start_time = time.time()
    with torch.no_grad():
        output = model(image_tensor)
        output = torch.sigmoid(output)
    inference_end_time = time.time()

    # Postprocessing
    postprocess_start_time = time.time()
    mask = output.squeeze(0).squeeze(0).cpu().numpy()
    mask = (mask > 0.5).astype(np.uint8) * 255
    mask_image = Image.fromarray(mask)

    combined = Image.new("RGB", (512 * 2, 512))
    combined.paste(image.resize((512, 512)), (0, 0))
    combined.paste(mask_image.convert("RGB"), (512, 0))
    combined.save("output.jpg")
    postprocess_end_time = time.time()

    # Calculate timing stats
    total_end_time = time.time()

    preprocess_time = preprocess_end_time - preprocess_start_time
    inference_time = inference_end_time - inference_start_time
    postprocess_time = postprocess_end_time - postprocess_start_time
    total_time = total_end_time - total_start_time

    # Print stats
    print("\nPrediction completed! Stats:")
    print(f"  Image Preprocessing Time: {preprocess_time:.4f} seconds")
    print(f"  Model Inference Time: {inference_time:.4f} seconds")
    print(f"  Postprocessing Time: {postprocess_time:.4f} seconds")
    print(f"  Total Prediction Time: {total_time:.4f} seconds")
    print("Prediction saved as output.jpg")


In [1]:
predict(model_path="/content/unet_model_final.pth", input_image_path="D:\Projects\Segmentation\007_Implementing_custom_Unet_training\Human-Segmentation-Dataset\Training_Images\7.jpg")

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\shiva\AppData\Local\Temp\ipykernel_18744\2639892166.py:1: SyntaxWarning: invalid escape sequence '\P'
  predict(model_path="/content/unet_model_final.pth", input_image_path="D:\Projects\Segmentation\007_Implementing_custom_Unet_training\Human-Segmentation-Dataset\Training_Images\7.jpg")
C:\Users\shiva\AppData\Local\Temp\ipykernel_18744\2639892166.py:1: SyntaxWarning: invalid escape sequence '\P'
  predict(model_path="/content/unet_model_final.pth", input_image_path="D:\Projects\Segmentation\007_Implementing_custom_Unet_training\Human-Segmentation-Dataset\Training_Images\7.jpg")


NameError: name 'predict' is not defined